In [ ]:
pip install google-generativeai requests beautifulsoup4 pandas

In [ ]:
pip install google-genai pydantic

In [ ]:
from pydantic import BaseModel, Field

In [ ]:
class TickertapeLink(BaseModel):
    """
    Defines the structure we expect from the AI model.
    """
    url: str = Field(description="The full URL to the stock's page on Tickertape.in")

In [94]:
# Correct way to read the Gemini API key from a file
import json
with open(r"C:\Users\Rohit Jonnadula\secret_key.txt", "r") as file:
    json_str = json.load(file)
    GEMINI_API_KEY = json_str.get("gemini")
    serpapi = json_str.get("serpapi")


In [ ]:
import google.generativeai as genai
import os

os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY  # or set this in your environment
genai.configure(api_key=os.environ["GEMINI_API_KEY"])

print("Gemini client initialized successfully.")

In [85]:
from google.genai import types
import re

def generate_link_with_gemini(ticker_symbol: str):
    # 3a. The prompt: The specific instruction for the AI.
    prompt = (
        f"get the url for nse ticker symbol '{ticker_symbol}' from tickertape."
        # f"The URL format is 'www.tickertape.in[TICKER_SYMBOL]'. "
        f"Only return the required JSON structure."
    )

    # 3b. The configuration: The rules for the output format.
    config = types.GenerateContentConfig(
        response_mime_type="application/json",
        response_schema=TickertapeLink, # Points to our Pydantic class
    )
    
    # ... move to Step 4 ...
    print(f"Calling Gemini API for ticker: {ticker_symbol}...")
    model = genai.GenerativeModel("gemini-2.5-flash")
    response = model.generate_content(prompt)

    # 5a. The response is a JSON string (e.g., '{"url": "..."}')
    print(f"Raw AI response text: {response.text}")


    raw_response = response.text
    # Remove ```json and ```
    if raw_response.startswith('```json'):
        clean_json = raw_response[7:].strip()  # Remove ```
    if raw_response.endswith('```'):
        clean_json = clean_json[:-3].strip()   # Remove ```
    data_object = TickertapeLink.model_validate_json(clean_json)


    # 5c. Access the specific attribute (the link) and assign it to a Python variable
    final_link_variable = data_object.url

    return final_link_variable

In [87]:
ticker_to_search = "pricolltd" # Example for an Indian stock
my_final_url = generate_link_with_gemini(ticker_to_search)

print("-" * 40)
print(f"The final link assigned to the variable 'my_final_url' is:")
print(f"\n>>>> {my_final_url} <<<<\n")

Calling Gemini API for ticker: pricolltd...
Raw AI response text: ```json
{
  "url": "https://www.tickertape.in/app/stocks/pricolltd"
}
```
----------------------------------------
The final link assigned to the variable 'my_final_url' is:

>>>> https://www.tickertape.in/app/stocks/pricolltd <<<<

Raw AI response text: ```json
{
  "url": "https://www.tickertape.in/app/stocks/pricolltd"
}
```
----------------------------------------
The final link assigned to the variable 'my_final_url' is:

>>>> https://www.tickertape.in/app/stocks/pricolltd <<<<



In [97]:
urls=[]

In [109]:
import requests
from typing import Optional

def get_tickertape_url(ticker_symbol: str, serpapi_key: str) -> Optional[str]:
    """
    Get exact Tickertape.in URL for NSE ticker using SerpAPI
    """
    # Multiple search strategies (broader → more reliable)
    search_queries = [
        f'site:tickertape.in "{ticker_symbol}" stock',  # Broadest
        f'site:tickertape.in/stocks/ {ticker_symbol}',  # Stocks section
        f'{ticker_symbol} site:tickertape.in',          # Ticker first
        f'{ticker_symbol} NSE tickertape',              # Natural search
    ]
    
    params_base = {
        "engine": "google",
        "api_key": serpapi_key,
        "num": 10
    }
    
    for i, query in enumerate(search_queries, 1):
        print(f"Try {i}: Searching '{query}'")
        params = params_base.copy()
        params["q"] = query
        
        try:
            response = requests.get("https://serpapi.com/search", params=params)
            results = response.json()
            
            for result in results.get("organic_results", []):
                url = result.get("link", "")
                print(f"  Checking: {url}")
                
                # Tickertape stock URL patterns
                if ("tickertape.in" in url and 
                    "/stocks/" in url and 
                    (ticker_symbol.lower() in url.lower() or 
                     ticker_symbol.upper() in result.get("title", "").upper())):
                    print(f"✅ FOUND: {url}")
                    return url
                    
        except Exception as e:
            print(f"  Error: {e}")
    
    print("❌ No Tickertape URL found in any search")
    return None

# Test
serpapi_key = serpapi
url = get_tickertape_url("bandhan bank", serpapi_key)
urls.append(url)
print(f"Final URL: {url}")


Try 1: Searching 'site:tickertape.in "bandhan bank" stock'
  Checking: https://www.tickertape.in/stocks/bandhan-bank-BANH
✅ FOUND: https://www.tickertape.in/stocks/bandhan-bank-BANH
Final URL: https://www.tickertape.in/stocks/bandhan-bank-BANH
  Checking: https://www.tickertape.in/stocks/bandhan-bank-BANH
✅ FOUND: https://www.tickertape.in/stocks/bandhan-bank-BANH
Final URL: https://www.tickertape.in/stocks/bandhan-bank-BANH


In [110]:
print (urls)

['https://www.tickertape.in/stocks/pricol-PRCO', 'https://www.tickertape.in/stocks/mps-MPSL', 'https://www.tickertape.in/stocks/mps-MPSL', 'https://www.tickertape.in/stocks/billionbrains-garage-ventures-GROWO', None, 'https://www.tickertape.in/stocks/sanghvi-movers-SNGM', 'https://www.tickertape.in/stocks/bandhan-bank-BANH']


In [ ]:
import pandas as pd
import requests
import google.generativeai as genai
from bs4 import BeautifulSoup
ticker_name = 'jkil'
url = f'https://www.tickertape.in/stocks/{ticker_name}'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
print(soup.prettify()[:1000])  # Print the first 1000 characters of the prettified HTML